In [2]:
GRAPHRAG_FOLDER = "/home/tagbin/Documents/graphRAG/ragtest/output/20240827-132848/artifacts"

In [3]:
import pandas as pd
from neo4j import GraphDatabase
import time

In [16]:
NEO4J_URI="neo4j+s://adbdb562.databases.neo4j.io:7687" # or neo4j+s://xxxx.databases.neo4j.io 
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="0g-L3_pMkT_pyOXjxmu0ZnKQjDRFCyWkUwLtMlNrqMA"
NEO4J_DATABASE="neo4j"

# Create a Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [17]:
with driver.session() as session:
    result = session.run("RETURN 1")
    print(result.single())


<Record 1=1>


In [18]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0,total, batch_size):
        batch = df.iloc[start: min(start+batch_size,total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in { time.time() - start_s} s.')    
    return total
    

In [19]:

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


In [20]:
doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_documents.parquet', columns=["id", "title"])
doc_df.head(2)

,id,title
0,f72da6ed3e5c1bd2a6420ffbd0e06130,State-Profile.txt
1,f0de8f06f883589bde0f1665933d5a49,AR-01-20-06-2008.txt


In [21]:
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'labels_added': 27, 'nodes_created': 27, 'properties_set': 54}
27 rows in 0.4226408004760742 s.


27

In [22]:
text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_text_units.parquet',
                          columns=["id","text","n_tokens","document_ids"])
text_df.head(2)

,id,text,n_tokens,document_ids
0,1a4f0d7d943fd50a162b9eca93dd288a,BIHAR’S AGRICULTURE DEVELOPMENT:\nOPPORTUNITIE...,1200,[0bb3a444c7698af65e69e5848d314a6e]
1,cb07ec9a460c593aa066f6ee8f4745c7,and internationally accepted hygienic supply ...,1200,[0bb3a444c7698af65e69e5848d314a6e]


In [23]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 294, 'relationships_created': 294, 'nodes_created': 294, 'properties_set': 882}
294 rows in 1.6061267852783203 s.


294

In [24]:
entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_entities.parquet',
                            columns=["name","type","description","human_readable_id","id","description_embedding","text_unit_ids"])
entity_df.head(2)

,name,type,description,human_readable_id,id,description_embedding,text_unit_ids
0,BIHAR,GEO,"Bihar, a densely populated and land-locked sta...",0,b45241d70f0e43fca764df95b2b81f77,"[0.011283425614237785, 0.012544927187263966, 0...","[00986a58e6cbca860c2c5535e5bf1011, 04ef372d8ba..."
1,RIVER GANGES,GEO,"A major river flowing through Bihar, dividing ...",1,4119fd06010c494caa07f439b333f4c5,"[0.020827675238251686, 0.00991102959960699, 0....",[1a4f0d7d943fd50a162b9eca93dd288a]


In [25]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 1000, 'relationships_created': 2649, 'nodes_created': 1000, 'properties_set': 4000}
{'_contains_updates': True, 'labels_added': 935, 'relationships_created': 1565, 'nodes_created': 935, 'properties_set': 3740}
1935 rows in 34.05861830711365 s.


1935

In [26]:
rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_relationships.parquet',
                         columns=["source","target","id","rank","weight","human_readable_id","description","text_unit_ids"])
rel_df.head(2)

,source,target,id,rank,weight,human_readable_id,description,text_unit_ids
0,BIHAR,RIVER GANGES,70890fae70dc43e59b7f33f364a25171,627,9.0,0,The River Ganges divides Bihar into North and ...,[1a4f0d7d943fd50a162b9eca93dd288a]
1,BIHAR,SUDHA,60e7c322568d45f88e7076d2a62caa04,627,1.0,1,Sudha is a notable exception among Bihar's sta...,[1a4f0d7d943fd50a162b9eca93dd288a]


In [27]:
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 1000, 'properties_set': 6000}
{'_contains_updates': True, 'relationships_created': 173, 'properties_set': 1038}
2173 rows in 2.7070324420928955 s.


2173

In [28]:
community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_communities.parquet', 
                     columns=["id","level","title","text_unit_ids","relationship_ids"])

community_df.head(2)


,id,level,title,text_unit_ids,relationship_ids
0,2,0,Community 2,"[00986a58e6cbca860c2c5535e5bf1011,04ef372d8ba0...","[70890fae70dc43e59b7f33f364a25171, 60e7c322568..."
1,15,0,Community 15,"[1a4f0d7d943fd50a162b9eca93dd288a,4bd45a16dbea...","[c154973be3cd4158b24ed7b54a2f38ae, 1548ac7f3b2..."


In [29]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURn count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 276, 'relationships_created': 7664, 'nodes_created': 276, 'properties_set': 828}
276 rows in 2.758641481399536 s.


276

In [30]:
community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_community_reports.parquet',
                               columns=["id","community","level","title","summary", "findings","rank","rank_explanation","full_content"])
community_report_df.head(2)

,id,community,level,title,summary,findings,rank,rank_explanation,full_content
0,a749e670-fef9-4fc0-bcd6-ef89f8cbd80f,275,5,Agricultural Innovation in Bihar: Centre of Ex...,This report focuses on the establishment of th...,"[{'examination': None, 'explanation': 'The Cen...",7.5,The impact severity rating is relatively high ...,# Agricultural Innovation in Bihar: Centre of ...
1,f6000d3f-d4ff-49ec-a69e-382e37879692,274,5,Agricultural Dynamics and Development in Bihar,This report delves into the agricultural lands...,"[{'examination': None, 'explanation': 'Krishi ...",7.5,The impact severity rating is relatively high ...,# Agricultural Dynamics and Development in Bih...


In [32]:
# import communities
community_statement = """
MERGE (c:__Community__ {id:value.id})
SET c += value {.community, .level, .title, .rank, .rank_explanation, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)


ConstraintError: {code: Neo.ClientError.Schema.ConstraintValidationFailed} {message: Node(2531) already exists with label `__Community__` and property `community` = '275'}

In [33]:
cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet'),
#                         columns=["id","text_unit_id"])
cov_df.head(2)
# Subject id do not match entity ids


FileNotFoundError: [Errno 2] No such file or directory: '/home/tagbin/Documents/graphRAG/ragtest/output/20240827-132848/artifacts/create_final_covariates.parquet'